In [1]:
import glob
import os
import re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = './data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if len(sentence.split()) >= 12:
        continue
    if sentence[-1] == ":":
        continue
       
    corpus.append(preprocess_sentence(sentence))

#print(corpus[:10])
tensor, tokenizer = tokenize(corpus)
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,
                                                          random_state=42,
                                                          test_size=0.2)

[[   2   52    5 ...    0    0    0]
 [   2   17 3181 ...    0    0    0]
 [   2   35    7 ...   41    3    0]
 ...
 [   2    5  120 ...    0    0    0]
 [   2  261  199 ...   12    3    0]
 [   2    7   36 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f66eef03f70>


In [4]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (121675, 14)
Target Train: (121675, 14)


In [5]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder = True)
test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder = True)

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.GRU(hidden_size, return_sequences=True)
        self.batch_norm_1 = tf.keras.layers.BatchNormalization()
        self.rnn_2 = tf.keras.layers.GRU(hidden_size, return_sequences=True)
        self.batch_norm_2 = tf.keras.layers.BatchNormalization()
        #self.rnn_3 = tf.keras.layers.GRU(hidden_size, dropout=0.2, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.batch_norm_1(out)
        out = self.rnn_2(out)
        out = self.batch_norm_2(out)
        #out = self.rnn_3(out)
        out = self.linear(out)
        
        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, validation_data = test_dataset, epochs=15)

Epoch 1/15
475/475 [==============================] - 83s 167ms/step - loss: 2.8906 - val_loss: 7.0300
Epoch 2/15
475/475 [==============================] - 79s 166ms/step - loss: 2.3248 - val_loss: 2.3616
Epoch 3/15
475/475 [==============================] - 79s 167ms/step - loss: 1.9914 - val_loss: 2.2889
Epoch 4/15
475/475 [==============================] - 79s 167ms/step - loss: 1.7550 - val_loss: 2.2557
Epoch 5/15
475/475 [==============================] - 79s 167ms/step - loss: 1.5834 - val_loss: 2.2519
Epoch 6/15
475/475 [==============================] - 79s 167ms/step - loss: 1.4545 - val_loss: 2.2629
Epoch 7/15
475/475 [==============================] - 79s 167ms/step - loss: 1.3542 - val_loss: 2.2745
Epoch 8/15
475/475 [==============================] - 79s 167ms/step - loss: 1.2732 - val_loss: 2.2976
Epoch 9/15
475/475 [==============================] - 80s 167ms/step - loss: 1.2080 - val_loss: 2.3274
Epoch 10/15
475/475 [==============================] - 79s 167ms/step - l

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.
        
        # 우리 모델이 새롭계 예측한 단어를 입력 문장의 뒤에 붙여줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면 while 루프를 돌면서 다음 단어를 예측해야합니다.
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break
    
    generated = ""
    
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '